# Script notebook - criar banco de dados SQL para consultas eleitorais
---
### Tabelas
- perfil de eleitorado
- locais de votação
- resultados eleitorais

In [1]:
ANO = '2024'
ESTADO = 'SP' # uppercase!

In [2]:
from tse import TseClient, df_to_sql, listar_tabelas

- Inicializando cliente

In [3]:
client = TseClient()

- Menu de datasets disponíveis

In [4]:
for i, dataset in enumerate(client.datasets):
    print(f'{i} -> {dataset}')

0 -> candidatos-1934
1 -> candidatos-1965
2 -> pesquisas-eleitorais-2020
3 -> eleitorado-1998
4 -> eleitorado-2024
5 -> comparecimento-e-abstencao-2024
6 -> resultados-2022
7 -> candidatos-2000
8 -> eleitorado-2016
9 -> prestacao-de-contas-eleitorais-2010
10 -> candidatos-1933
11 -> prestacao-de-contas-partidarias-2015
12 -> resultados-2010
13 -> prestacao-de-contas-partidarias-2023
14 -> eleitorado-2020
15 -> resultados-1996
16 -> pesquisas-eleitorais-2016
17 -> resultados-2022-boletim-de-urna
18 -> resultados-1947
19 -> resultados-2008
20 -> resultados-2024-arquivos-transmitidos-para-totalizacao
21 -> resultados-2012
22 -> eleitorado-2014
23 -> mesarios-2022
24 -> pesquisas-eleitorais-2014
25 -> dadosabertos-tse-jus-br-dataset-prestacao-de-contas-eleitorais-2022
26 -> correspondencia-entre-numero-interno-e-modelo-da-urna-1
27 -> prestacao-de-contas-eleitorais-2018
28 -> resultados-2016-correspondencias-esperadas-e-efetivadas-1-turno
29 -> prestacao-de-contas-partidarias-2022
30 -> ca

## Escolhendo os datasets:
### eleitorado-2024
- perfil_eleitor_secao_2024_RJ: informacoes sobre perfil dos eleitores por secao no RJ
- eleitorado_local_votacao_2024: informacoes sobre locais de votacao

### resultados-2024
- votacao_secao_2024_RJ: informacoes sobre resultado das votacoes por secao no RJ


In [5]:
data = {} # armazena dataframes das tabelas selecionadas

- Dataset: perfil do eleitorado

In [6]:
perfil_eleitorado = client.find_dataset(f'eleitorado-{ANO}', pattern=f'{ESTADO}')
perfil_eleitorado

{'https://cdn.tse.jus.br/estatistica/sead/odsele/perfil_eleitor_secao/perfil_eleitor_secao_2024_SP.zip'}

- Dataset: locais de votacao (BRASIL)

In [7]:
locais = client.find_dataset(f'eleitorado-{ANO}', pattern='local')
locais

{'https://cdn.tse.jus.br/estatistica/sead/odsele/eleitorado_locais_votacao/eleitorado_local_votacao_2024.zip'}

- Dataset: resultados da votação RJ

In [8]:
resultados = client.find_dataset(f'resultados-{ANO}', pattern=f'{ESTADO}')
resultados

{'https://cdn.tse.jus.br/estatistica/sead/odsele/relatorio_resultado_totalizacao/Relatorio_Resultado_Totalizacao_2024_SP.zip',
 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2024_SP.zip'}

- Download e carregamento de cada dataset em um dataframe

In [9]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(perfil_eleitorado)
    )
)

Files downloaded to data\perfil_eleitor_secao_2024_SP


In [10]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(locais)
    )
)

Files downloaded to data\eleitorado_local_votacao_2024


In [11]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(resultados)
    )
)

Files downloaded to data\Relatorio_Resultado_Totalizacao_2024_SP
Files downloaded to data\votacao_secao_2024_SP


In [12]:
data.keys()

dict_keys(['perfil_eleitor_secao_2024_SP', 'eleitorado_local_votacao_2024', 'votacao_secao_2024_SP'])

- Limpeza de algumas colunas

In [13]:
resultados_df = data.get(f'votacao_secao_{ANO}_{ESTADO}')

In [14]:
resultados_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_LOCAL_VOTACAO',
       'SQ_CANDIDATO', 'NM_LOCAL_VOTACAO', 'DS_LOCAL_VOTACAO_ENDERECO'],
      dtype='object')

In [15]:
resultados_columns = [
    'ANO_ELEICAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'NR_TURNO', 
    'CD_CARGO', 
    'DS_CARGO',
    'NR_VOTAVEL',
    'NM_VOTAVEL',
    'QT_VOTOS'
]
resultados_df = resultados_df[resultados_columns]

In [16]:
resultados_df.head()

,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NR_TURNO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS
0,2024,SP,67890,315,2097,324,1,13,Vereador,22322,CLAUDIO DO NASCIMENTO TORRES,1
1,2024,SP,67890,276,1058,120,1,13,Vereador,22322,CLAUDIO DO NASCIMENTO TORRES,1
2,2024,SP,62413,16,1104,413,1,13,Vereador,45000,TASSIA CAROLINA PEREIRA FERREIRA,3
3,2024,SP,65692,58,1988,164,1,11,Prefeito,95,VOTO BRANCO,5
4,2024,SP,63134,388,1058,283,1,13,Vereador,55777,RONALDO DE SOUZA,6


- Limpeza de algumas colunas

In [17]:
locais_df = data.get(f'eleitorado_local_votacao_{ANO}')

In [18]:
locais_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'AA_ELEICAO', 'DT_ELEICAO', 'DS_ELEICAO',
       'NR_TURNO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA',
       'NR_SECAO', 'CD_TIPO_SECAO_AGREGADA', 'DS_TIPO_SECAO_AGREGADA',
       'NR_SECAO_PRINCIPAL', 'NR_LOCAL_VOTACAO', 'NM_LOCAL_VOTACAO',
       'CD_TIPO_LOCAL', 'DS_TIPO_LOCAL', 'DS_ENDERECO', 'NM_BAIRRO', 'NR_CEP',
       'NR_TELEFONE_LOCAL', 'NR_LATITUDE', 'NR_LONGITUDE',
       'CD_SITU_LOCAL_VOTACAO', 'DS_SITU_LOCAL_VOTACAO', 'CD_SITU_ZONA',
       'DS_SITU_ZONA', 'CD_SITU_SECAO', 'DS_SITU_SECAO', 'CD_SITU_LOCALIDADE',
       'DS_SITU_LOCALIDADE', 'CD_SITU_SECAO_ACESSIBILIDADE',
       'DS_SITU_SECAO_ACESSIBILIDADE', 'QT_ELEITOR_SECAO',
       'QT_ELEITOR_ELEICAO_FEDERAL', 'QT_ELEITOR_ELEICAO_ESTADUAL',
       'QT_ELEITOR_ELEICAO_MUNICIPAL', 'NR_LOCAL_VOTACAO_ORIGINAL',
       'NM_LOCAL_VOTACAO_ORIGINAL', 'DS_ENDERECO_LOCVT_ORIGINAL'],
      dtype='object')

In [19]:
locais_columns = [
    'AA_ELEICAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO',
    'NM_MUNICIPIO',
    'NM_BAIRRO',
    'NR_LATITUDE',
    'NR_LONGITUDE',
    'QT_ELEITOR_SECAO'
]
locais_df = locais_df[locais_columns]

In [20]:
locais_df.head()

,AA_ELEICAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NM_MUNICIPIO,NM_BAIRRO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITOR_SECAO
0,2024,BA,39730,104,1147,62,LAPÃO,POVOADO DE CASAL I,-11.516413,-41.802394,262
1,2024,BA,39730,104,1457,181,LAPÃO,CENTRO,-11.379024,-41.835959,307
2,2024,SC,83798,36,1147,103,VIDEIRA,UNIVERSITÁRIO,-26.992234,-51.176671,388
3,2024,SC,83798,36,1015,95,VIDEIRA,CENTRO,-27.002251,-51.157982,271
4,2024,SP,72451,147,1074,178,VOTUPORANGA,VILA PAES,-20.414684,-49.971817,250


- Limpeza de algumas colunas

In [21]:
perfil_df = data.get(f'perfil_eleitor_secao_{ANO}_{ESTADO}')

In [22]:
perfil_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
       'NM_LOCAL_VOTACAO', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'CD_RACA_COR',
       'DS_RACA_COR', 'CD_IDENTIDADE_GENERO', 'DS_IDENTIDADE_GENERO',
       'CD_QUILOMBOLA', 'DS_QUILOMBOLA', 'CD_INTERPRETE_LIBRAS',
       'DS_INTERPRETE_LIBRAS', 'QT_ELEITORES_PERFIL', 'QT_ELEITORES_BIOMETRIA',
       'QT_ELEITORES_DEFICIENCIA', 'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [23]:
perfil_columns = [
    'ANO_ELEICAO',
    'DT_GERACAO',
    'HH_GERACAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'DS_GENERO', 
    'DS_ESTADO_CIVIL',
    'DS_FAIXA_ETARIA',
    'DS_GRAU_ESCOLARIDADE',
    'DS_RACA_COR',
    'QT_ELEITORES_PERFIL'   
]
perfil_df = perfil_df[perfil_columns]

In [24]:
perfil_df.head()

,ANO_ELEICAO,DT_GERACAO,HH_GERACAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,DS_GENERO,DS_ESTADO_CIVIL,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE,DS_RACA_COR,QT_ELEITORES_PERFIL
0,2024,13/09/2024,13:12:31,SP,63177,39,1937,66,MASCULINO,VIÚVO,60 a 64 anos,ENSINO FUNDAMENTAL INCOMPLETO,NÃO INFORMADO,1
1,2024,13/09/2024,13:12:31,SP,61891,17,1066,80,FEMININO,CASADO,50 a 54 anos,SUPERIOR COMPLETO,Branca,1
2,2024,13/09/2024,13:12:31,SP,67059,165,1023,49,FEMININO,SOLTEIRO,20 anos,SUPERIOR INCOMPLETO,NÃO INFORMADO,1
3,2024,13/09/2024,13:12:31,SP,62138,386,1260,108,FEMININO,SOLTEIRO,21 a 24 anos,SUPERIOR INCOMPLETO,NÃO INFORMADO,1
4,2024,13/09/2024,13:12:31,SP,70572,383,1120,222,FEMININO,CASADO,35 a 39 anos,ENSINO FUNDAMENTAL COMPLETO,NÃO INFORMADO,1


- Finalmente: conversão para SQL database

In [25]:
df_to_sql(
    df=locais_df, 
    table_name=f'locais', 
    dbname=f'eleicoes_{ANO}.sql')

df_to_sql(
    df=resultados_df,
    table_name='resultados',
    dbname=f'{ESTADO.lower()}_{ANO}.sql')

df_to_sql(
    df=perfil_df,
    table_name='eleitorado',
    dbname=f'{ESTADO.lower()}_{ANO}.sql')

Tabela "locais" foi criada no banco de dados "databases\eleicoes_2024.sql".
Tabela "resultados" foi criada no banco de dados "databases\sp_2024.sql".
Tabela "eleitorado" foi criada no banco de dados "databases\sp_2024.sql".


'eleitorado'

In [26]:
listar_tabelas(f'databases/{ESTADO.lower()}_{ANO}.sql')

Tabelas no banco de dados:
- resultados
- eleitorado


['resultados', 'eleitorado']

In [27]:
listar_tabelas(f'databases/eleicoes_{ANO}.sql')

Tabelas no banco de dados:
- locais


['locais']

- Removendo arquivos temporários

In [28]:
TseClient.remove_data()

'Diretório data esvaziado'

: 